In [1]:
import pandas as pd
import numpy as np
from scipy.special import gamma
from math import log
from scipy.optimize import minimize

In [2]:
samples = pd.read_csv('D:/thesis_data/VEG_INDICES/samples/stratified/campestre/20m/FC_20170112_20m_patches.csv', sep=',')

In [3]:
dprvi_samples = samples['dprvi_patch_0'].values

In [4]:
def g0_estimator_moments(samples, L):

    # This function derives the parameters alpha and gamma from a G0 distribution by the moments method
    # This method assumes the Number of Looks (L) as known (for Sentinel-1: L = 4)

    m1 = np.mean(samples)
    m2 = np.mean(samples**2)

    m212 = m2/(m1**2)

    a = -2-(L+1)/(L*m212)
    g  = m1*(2+(L+1)/(L*m212))

    return round(a, 6), round(g, 6)

In [84]:
alpha0, gamma0 = g0_estimator_moments(dprvi_samples,  L=4)
print(f'a = {alpha0}, g = {gamma0}')

a = -3.213836, g = 1.98798


In [107]:
def loglikelihood_estimator(chute_inicial):

    # This function derives the parameters alpha and gamma from a G0 distribution by the maximum likelihhod method
    # The input for this function is the alpha and gamma estimated previously by the moments method
    # This method assumes the Number of Looks (L) as known (for Sentinel-1: L = 4)
    # Samples is an array containing the pixels values of a given region in the SAR image

    p_alpha = chute_inicial[0]
    p_gamma = chute_inicial[1]

    samples = dprvi_samples
    n = len(samples) # Number of samples

    return n * (gamma(4 - p_alpha) - p_alpha * np.log(p_gamma) - gamma(-p_alpha)) + (p_alpha-4) * sum(np.log(p_gamma + samples * 4))

In [108]:
chute_inicial = [alpha0, gamma0]

In [111]:
results = minimize(loglikelihood_estimator, x0=chute_inicial, method='BFGS', options={'maxiter': 500},tol=1e-8)

In [113]:
results.x

array([5.72444672e-01, 1.78902018e+07])